In [ ]:
from collections import Counter
import pickle
import numpy as np
from openbackdoor import load_dataset
import random
from utils import setup_seed, certified, majority
import os

data = "hsol"
path = f"certified/{data}-noise1-clean-0.0cert"
if data == "agnews":
    C = 4
else:
    C = 2
group = 7
preds = []
for i in range(group):
    with open(f"{path}{group}/42/clean_{i}.pkl", "rb") as f:
        pred = pickle.load(f)
    preds.append(pred)
    
setup_seed(42)
dataset = load_dataset(name=data)
n = len(dataset["train"])
gold = [x[1] for x in dataset["test"]]


In [ ]:
print(certified(preds, gold, C=C, target_label=1))

In [ ]:
def individual(preds, gold, p):
    np_res = np.array(preds)
    acc = 0
    difs = []
    tot = 0
    for i in range(np_res.shape[1]):
        if gold[i]==1:
            continue
        tot += 1
        cnt = np.zeros(C)
        for x in np_res[:, i]:
            cnt[x]-=1
        idxs = cnt.argsort(kind="stable")
        x, y = idxs[0], -cnt[idxs[0]]
        xx, yy = idxs[1], -cnt[idxs[1]]        

        dif = int((y-(yy+int(xx<x)))/2)
        difs.append(dif)
        if x==gold[i] and dif>=p:
            acc +=1
        #print(cnt, gold[i], dif)
        #if i>20:
        #    break
    return acc/tot

lis = []
for i in range(1,4):
    lis.append(individual(preds, gold, i))
print(lis)


In [ ]:
from itertools import combinations
from collections import Counter
from sklearn.metrics import accuracy_score
def certified2(preds, labels, C=2, target_label=None):
    # C is number of classes
    preds = np.array(preds) # Group * N
    labels = np.array(labels) # N
    if target_label is not None:
        target = labels!=target_label
        print(len(target), sum(target))
        preds = preds[:, target]
        labels = labels[target]
        print(preds.shape)
        
    final_pred = majority(preds, C=C)
    correct = (labels == final_pred)
    n_correct = sum(correct)
    n_wrong = len(labels) - n_correct
    lis_cacc = [n_correct/(n_correct+n_wrong)]
    m = len(preds)
    for i in range(1, m//2+1): # number of backdoored groups
        cacc = 1
        for lis in combinations(range(m), i): # iterate all combinations
            s = 0
            for k, x in enumerate(preds.transpose()):
                cnt = np.zeros(C)
                for xx in x:
                    cnt[xx]-=1
                idxs = cnt.argsort(kind="stable")
                a = idxs[0]
                U = -cnt[a]
                for j in lis:
                    if x[j]==a:
                        U-=1
                L = 0
                for b in range(C):
                    if b!=a:
                        r = 0
                        for j in lis:
                            if x[j]!=b:
                                r+=1
                        L = max(L, -cnt[b]+int(a>b)+r)
                if a==labels[k] and U>=L:
                    s+=1
            cacc = min(cacc, s/len(labels))
        lis_cacc.append(cacc)        
    return lis_cacc
print(certified2(preds, gold, C, target_label=1))